### Synthetic dataset of Pakistani ID card signatures

In [25]:
from PIL import Image
import matplotlib.pyplot as plt 
import cv2
import numpy as np
import os
import json

In [66]:
file_name = {}
for _, _, files in os.walk ("./dataset/", followlinks=False):
    file_name['signature_data'] = files
# Writing Json file
with open("./dataset/sample.json", "w") as outfile:
    json.dump(file_name, outfile)
# print(json_object)


In [52]:
# Function to make sign transparent
def make_transparent(im_array):
    
    gray = cv2.cvtColor(im_array, cv2.COLOR_BGR2GRAY)

    mask = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY)[1]
    mask = 255 - mask

    kernel = np.ones((3,3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    mask = cv2.GaussianBlur(mask, (0,0), sigmaX=2, sigmaY=2, borderType = cv2.BORDER_DEFAULT)

    mask = (2*(mask.astype(np.float32))-255.0).clip(0,255).astype(np.uint8)
    result = im_array.copy()

    result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
    result[:, :, 3] = mask
    return result




In [71]:
def merge_two_images(id_card, signature):

    # Resize the boy image to fit the road image
    signature = cv2.resize(signature, (int(signature.shape[1] * 0.5), int(signature.shape[0] * 0.5)))


    # Extract the alpha channel of the boy image
    alpha = signature[:, :, 3] / 255.0

    # Convert the boy image to BGR without the alpha channel
    signature = signature[:, :, :3]


    # Resize the alpha channel to the same size as the boy image
    alpha = cv2.resize(alpha, signature.shape[:2][::-1])

    # Calculate the foreground and background by multiplying the images and alpha channel
    foreground = alpha[..., None] * signature
    background = (1 - alpha[..., None]) * id_card[:signature.shape[0], :signature.shape[1]]

    # Combine the foreground and background using the cv2.addWeighted function
    result = cv2.addWeighted(background.astype(np.uint8), 1, foreground.astype(np.uint8), 1, 0)

    return result




In [58]:
for items in file_name.values():
    # image_name = next(iter(value))
    
    for file in items:
        my_file  = cv2.imread(f'./dataset/{file}')

        transparent = make_transparent(my_file)
        
        if not os.path.exists('transparent_dataset'):
            os.makedirs('transparent_dataset')
        img_name = file.split('.')[0]
        cv2.imwrite(f'transparent_dataset/{img_name}.png', transparent)


In [93]:
#Reading files from transparent directory
for _, _, files in os.walk ("./transparent_dataset/", followlinks=False):

    for image in files: 
        id_card = cv2.imread('id_card_template.jpg')
        signature = cv2.imread(f"./transparent_dataset/{image}", cv2.IMREAD_UNCHANGED)
        
        final_result = merge_two_images(id_card, signature)
        
        
        
        if not os.path.exists('final_results'):
            os.makedirs('final_results')
        img_name = image.split('.')[0]
        cv2.imwrite(f'./final_results/{img_name}_card.jpg', final_result)
        
        
        
        
    

        